In [4]:
!pip3 install us

In [5]:
!pip3 install geopandas

In [6]:
import requests
import pandas as pd
from us import states
import geopandas as gpd
import warnings; warnings.filterwarnings('ignore')

In [7]:
path = '/content/drive/MyDrive/Milestone II/Data Merge'

# **1. FIPS Data**



# **FIPS Data for State and County Code**

[Fips Data - Transportation.gov](https://data.transportation.gov/Railroads/State-County-and-City-FIPS-Reference-Table/eek5-pv8d/data_preview)

In [8]:
fips_path = f"{path}/data/fips/FIPS_Reference_Table_20251006.csv"
fips = pd.read_csv(fips_path, dtype=str, low_memory=False)
fips[ fips["State Code"] == 'VA']
#fips.head()

,State Name,County Name,City Name,State Code,State FIPS Code,County Code,StCnty FIPS Code,City Code,StCntyCity FIPS Code
35592,VIRGINIA,ACCOMACK,ACCOMAC,VA,51,C001,51001,0020,510010020
35593,VIRGINIA,ACCOMACK,ASSAWOMAN,VA,51,C001,51001,0113,510010113
35594,VIRGINIA,ACCOMACK,ATLANTIC,VA,51,C001,51001,0116,510010116
35595,VIRGINIA,ACCOMACK,BELLE HAVEN,VA,51,C001,51001,0140,510010140
35596,VIRGINIA,ACCOMACK,BLOXOM,VA,51,C001,51001,0198,510010198
...,...,...,...,...,...,...,...,...,...
40160,VIRGINIA,VIRGINIA BEACH,OCEANA NAVAL AIR STATI,VA,51,C810,51810,1822,518101822
40179,VIRGINIA,ARLINGTON,"PENTAGON, ARLINGTON",VA,51,C013,51013,1906,510131906
40181,VIRGINIA,BUCHANAN,PILGRIMS KNOB,VA,51,C027,51027,1936,510271936
40195,VIRGINIA,LOUDOUN,POTOMAC FALLS,VA,51,C107,51107,1982,511071982


# **Data Cleansing & Renaming**

In [9]:
fips.rename(columns={
    "State FIPS Code": "state_fips",
    "State Name": "state_name",
    "State Code": "state_abbr",
    "StCnty FIPS Code": "county_fips",
    "County Name" : "county_name",
    "City Name": "city_name",
    "City Code": "city_fips",
    "StCntyCity FIPS Code": "state_county_city_fips"
}, inplace=True)

fips = fips.drop(['city_fips', 'state_county_city_fips', 'city_name'], axis=1)
fips = fips.drop_duplicates()
fips.head()

,state_name,county_name,state_abbr,state_fips,County Code,county_fips
0,ALABAMA,AUTAUGA,AL,01,C001,01001
6,ALABAMA,BALDWIN,AL,01,C003,01003
30,ALABAMA,BARBOUR,AL,01,C005,01005
38,ALABAMA,BIBB,AL,01,C007,01007
47,ALABAMA,BLOUNT,AL,01,C009,01009


# **2. Census Data**

[Census Data](https://www.census.gov/data/developers/data-sets/popest-popproj.html)

In [10]:
API_KEY = "42009985e19aff365d6a28751d9b68e5931b9dd4"
vars = ["NAME", "B01003_001E", "B19013_001E"]
url = (
    f"https://api.census.gov/data/2023/acs/acs5?"
    f"get={','.join(vars)}&for=county:*&key={API_KEY}"
)
response = requests.get(url)
response.raise_for_status()
data = response.json()
headers = data[0]
rows = data[1:]
census_df = pd.DataFrame(rows, columns=headers)
census_df.head()

,NAME,B01003_001E,B19013_001E,state,county
0,"Autauga County, Alabama",59285,69841,01,001
1,"Baldwin County, Alabama",239945,75019,01,003
2,"Barbour County, Alabama",24757,44290,01,005
3,"Bibb County, Alabama",22152,51215,01,007
4,"Blount County, Alabama",59292,61096,01,009


# **Data Cleansing and Renaming**

In [11]:
census_df.rename(columns={
    "NAME": "state_name",
    "B01003_001E": "population",
    "B19013_001E": "median_income",
    "state": "state_fips"
}, inplace=True)
census_df["population"] = census_df["population"].astype(float)
census_df["median_income"] = census_df["median_income"].astype(float)
census_df["county_fips"] = census_df["state_fips"] + census_df["county"]
census_df.head()

,state_name,population,median_income,state_fips,county,county_fips
0,"Autauga County, Alabama",59285.0,69841.0,01,001,01001
1,"Baldwin County, Alabama",239945.0,75019.0,01,003,01003
2,"Barbour County, Alabama",24757.0,44290.0,01,005,01005
3,"Bibb County, Alabama",22152.0,51215.0,01,007,01007
4,"Blount County, Alabama",59292.0,61096.0,01,009,01009


# **Merging FIPS and Census Data**

In [12]:
c_df = pd.merge( census_df, fips, how="left", on="county_fips")
c_df = c_df.drop(['state_name_y', 'state_fips_y'], axis=1)
c_df.rename(columns={
    "state_name_x": "state_name",
    "state_fips_x": "state_fips"
}, inplace=True)

c_df.head()

,state_name,population,median_income,state_fips,county,county_fips,county_name,state_abbr,County Code
0,"Autauga County, Alabama",59285.0,69841.0,01,001,01001,AUTAUGA,AL,C001
1,"Baldwin County, Alabama",239945.0,75019.0,01,003,01003,BALDWIN,AL,C003
2,"Barbour County, Alabama",24757.0,44290.0,01,005,01005,BARBOUR,AL,C005
3,"Bibb County, Alabama",22152.0,51215.0,01,007,01007,BIBB,AL,C007
4,"Blount County, Alabama",59292.0,61096.0,01,009,01009,BLOUNT,AL,C009


# **Export to CSV**

In [13]:
c_df.to_csv(f"{path}/data/census/census_data_2023.csv")

# **3. Alternative Fuel Stations Data**

[AFS Data](https://geodata.bts.gov/datasets/usdot::alternative-fueling-stations/explore?location=17.881155%2C66.031405%2C3.46&showTable=true)

In [14]:
df = pd.read_csv(f"{path}/data/afs/Alternative_Fueling_Stations.csv", dtype=str)
df.head()

,OBJECTID,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,...,bd_blends_fr,groups_with_access_code_fr,ev_pricing_fr,federal_agency_id,federal_agency_code,federal_agency_name,ev_network_ids_station,ev_network_ids_posts,x,y
0,1,private,NaN,NaN,NaN,8/12/2025 12:00:00 AM,NaN,CNG,Private,17,...,NaN,Privé,NaN,NaN,NaN,NaN,NaN,NaN,-9603200.85,3811702.8042
1,2,private,NaN,GOVERNMENT,NaN,2/12/2024 12:00:00 AM,NaN,CNG,Private - Government only,45,...,NaN,Privé - Réservé au gouvernement,NaN,NaN,NaN,NaN,NaN,NaN,-9391742.798,4004914.0184
2,3,public,24 hours daily,CREDIT_CARD_ALWAYS,CREDIT M V Voyager,6/6/2025 12:00:00 AM,NaN,CNG,Public - Credit card at all times,73,...,NaN,Public - Carte de crédit en tout temps,NaN,NaN,NaN,NaN,NaN,NaN,-10505814.5696,4213214.0416
3,4,public,24 hours daily; call 866-809-4869 for Clean En...,CREDIT_CARD_ALWAYS,A CleanEnergy Comdata D FuelMan M V Voyager Wr...,12/10/2024 12:00:00 AM,NaN,CNG,Public - Credit card at all times,81,...,NaN,Public - Carte de crédit en tout temps,NaN,NaN,NaN,NaN,NaN,NaN,-7906639.2236,5217274.8305
4,5,public,24 hours daily; call 866-809-4869 for Clean En...,CREDIT_CARD_ALWAYS,CleanEnergy D FleetOne FuelMan M V Voyager Wri...,12/10/2024 12:00:00 AM,NaN,CNG,Public - Credit card at all times,84,...,NaN,Public - Carte de crédit en tout temps,NaN,NaN,NaN,NaN,NaN,NaN,-7910847.4782,5220057.1162


# **AFS Data Cleansing & Renaming**

In [15]:
# Keep ONLY Electric Fuel Stations
ev_df = df[df["fuel_type_code"] == "ELEC"]


In [16]:
cols = [
    "id", "date_last_confirmed", "station_name", "city", "state", "street_address", "zip", "latitude", "longitude",
    "ev_connector_types", "ev_level1_evse_num", "ev_level2_evse_num",
    "ev_dc_fast_num",  "ev_pricing", "access_days_time", "facility_type",
    "ev_network"
]
ev_df["date_last_confirmed"] = pd.to_datetime( ev_df["date_last_confirmed"] )
ev_df = ev_df[cols]
ev_df.head()

,id,date_last_confirmed,station_name,city,state,street_address,zip,latitude,longitude,ev_connector_types,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_num,ev_pricing,access_days_time,facility_type,ev_network
124,1517,2023-09-14,LADWP - Truesdale Center,Sun Valley,CA,11797 Truesdale St,91352,34.2483191527193,-118.387971374344,"[""CHADEMO"",""J1772"",""J1772COMBO""]",NaN,57,2,NaN,Fleet use only,UTILITY,SHELL_RECHARGE
125,1523,2023-01-10,Los Angeles Convention Center,Los Angeles,CA,1201 S Figueroa St,90015,34.040539,-118.271387,"[""""J1772""""]",NaN,7,NaN,Free; parking fee,5:30am-9pm; pay lot,PARKING_GARAGE,Non-Networked
126,1525,2023-09-14,LADWP - John Ferraro Building,Los Angeles,CA,111 N Hope St,90012,34.059133,-118.248589,"[""CHADEMO"",""J1772"",""J1772COMBO""]",NaN,338,12,NaN,For fleet and employee use only,UTILITY,Non-Networked
127,1531,2024-01-09,LADWP - Haynes Power Plant,Long Beach,CA,6801 E 2nd St,90803,33.759802,-118.096665,"[""CHADEMO"",""J1772"",""J1772COMBO""]",NaN,19,1,NaN,Fleet use only,UTILITY,Non-Networked
128,1552,2024-01-09,LADWP - Harbor Generating Station,Wilmington,CA,161 N Island Ave,90744,33.770508,-118.265628,"[""""J1772""""]",NaN,10,NaN,NaN,Fleet use only,UTILITY,Non-Networked


# **4. Zip Code Fips**

[zip fips](https://the-examples-book.com/projects/data-sets/fips)

In [17]:
zip_df = pd.read_csv(f"{path}/data/fips/ZIP-COUNTY-FIPS_2017-06.csv")
zip_df['STCOUNTYFP'] = zip_df['STCOUNTYFP'].astype(str)
zip_df['ZIP'] = zip_df['ZIP'].astype(str)
zip_df['county_fips'] = zip_df['STCOUNTYFP'].str.zfill(5)
zip_df["zip"] = zip_df["ZIP"]
zip_df.head()
#zip_df.info()

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP,county_fips,zip
0,36003,Autauga County,AL,1001,H1,01001,36003
1,36006,Autauga County,AL,1001,H1,01001,36006
2,36067,Autauga County,AL,1001,H1,01001,36067
3,36066,Autauga County,AL,1001,H1,01001,36066
4,36703,Autauga County,AL,1001,H1,01001,36703


# **Merging ZIP Fips with AFS Data**

In [18]:
ev_zip_df = pd.merge( ev_df, zip_df, how="left", on="zip")
ev_zip_df.head()

,id,date_last_confirmed,station_name,city,state,street_address,zip,latitude,longitude,ev_connector_types,...,ev_pricing,access_days_time,facility_type,ev_network,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP,county_fips
0,1517,2023-09-14,LADWP - Truesdale Center,Sun Valley,CA,11797 Truesdale St,91352,34.2483191527193,-118.387971374344,"[""CHADEMO"",""J1772"",""J1772COMBO""]",...,NaN,Fleet use only,UTILITY,SHELL_RECHARGE,91352,Los Angeles County,CA,6037,H1,06037
1,1523,2023-01-10,Los Angeles Convention Center,Los Angeles,CA,1201 S Figueroa St,90015,34.040539,-118.271387,"[""""J1772""""]",...,Free; parking fee,5:30am-9pm; pay lot,PARKING_GARAGE,Non-Networked,90015,Los Angeles County,CA,6037,H1,06037
2,1525,2023-09-14,LADWP - John Ferraro Building,Los Angeles,CA,111 N Hope St,90012,34.059133,-118.248589,"[""CHADEMO"",""J1772"",""J1772COMBO""]",...,NaN,For fleet and employee use only,UTILITY,Non-Networked,90012,Los Angeles County,CA,6037,H1,06037
3,1531,2024-01-09,LADWP - Haynes Power Plant,Long Beach,CA,6801 E 2nd St,90803,33.759802,-118.096665,"[""CHADEMO"",""J1772"",""J1772COMBO""]",...,NaN,Fleet use only,UTILITY,Non-Networked,90803,Los Angeles County,CA,6037,H1,06037
4,1552,2024-01-09,LADWP - Harbor Generating Station,Wilmington,CA,161 N Island Ave,90744,33.770508,-118.265628,"[""""J1772""""]",...,NaN,Fleet use only,UTILITY,Non-Networked,90744,Los Angeles County,CA,6037,H1,06037


# **Merging Census with AFS Data**

In [19]:
#census_df = pd.read_csv(f"{path}/data/census/census_data_2023.csv")
census_df = c_df.copy()
census_df['county_fips'] = census_df['county_fips'].astype(str)
census_df['county_fips'] = census_df['county_fips'].str.zfill(5)

ev_zip_census_df = pd.merge( ev_zip_df, census_df, how="left", on="county_fips")
ev_zip_census_df = ev_zip_census_df.drop(['County Code', 'county_fips', 'state_fips', 'county', 'county_name', 'STCOUNTYFP', 'CLASSFP', 'state_abbr'  ], axis=1)
ev_zip_census_df['Year'] = ev_zip_census_df['date_last_confirmed'].dt.year
#ev_census_df['month'] = ev_census_df['date_last_confirmed'].dt.to_period('M').astype(str)
ev_zip_census_df['Year_Month'] = ev_zip_census_df['date_last_confirmed'].dt.strftime('%b, %Y')
ev_zip_census_df['Month'] = ev_zip_census_df['date_last_confirmed'].dt.strftime('%m')
ev_zip_census_df = ev_zip_census_df.drop(columns=["zip", "state"], axis = 1)
ev_zip_census_df[["id","date_last_confirmed","access_days_time", "Year", 'Year_Month', "Month"]].head()


,id,date_last_confirmed,access_days_time,Year,Year_Month,Month
0,1517,2023-09-14,Fleet use only,2023.0,"Sep, 2023",09
1,1523,2023-01-10,5:30am-9pm; pay lot,2023.0,"Jan, 2023",01
2,1525,2023-09-14,For fleet and employee use only,2023.0,"Sep, 2023",09
3,1531,2024-01-09,Fleet use only,2024.0,"Jan, 2024",01
4,1552,2024-01-09,Fleet use only,2024.0,"Jan, 2024",01


In [20]:
ev_zip_census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104566 entries, 0 to 104565
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   id                   104566 non-null  object        
 1   date_last_confirmed  103907 non-null  datetime64[ns]
 2   station_name         104565 non-null  object        
 3   city                 104562 non-null  object        
 4   street_address       104525 non-null  object        
 5   latitude             104566 non-null  object        
 6   longitude            104566 non-null  object        
 7   ev_connector_types   102889 non-null  object        
 8   ev_level1_evse_num   752 non-null     object        
 9   ev_level2_evse_num   85362 non-null   object        
 10  ev_dc_fast_num       19159 non-null   object        
 11  ev_pricing           19937 non-null   object        
 12  access_days_time     93641 non-null   object        
 13  facility_type 

In [21]:
ev_census_df = ev_zip_census_df.copy()
group_cols = [
    'station_name', 'city', 'street_address',
    'latitude', 'longitude', 'ev_connector_types', 'ev_level1_evse_num',
    'ev_level2_evse_num', 'ev_dc_fast_num', 'ev_pricing', 'facility_type',
    'ev_network', 'ZIP', 'COUNTYNAME', 'STATE', 'state_name',
    'Year', 'Month'
]
ev_census_df['population']    = pd.to_numeric(ev_census_df['population'], errors='coerce')
ev_census_df['median_income'] = pd.to_numeric(ev_census_df['median_income'], errors='coerce')

agg_dict = {
    'population': 'mean',       # population is regional and avoids overcounting
    'median_income': 'mean',    # median income per region
    'id': 'count'               # count of records (number of stations/entries)
}

ev_census_df = (
    ev_census_df.groupby(group_cols, dropna=False)
      .agg(agg_dict)
      .reset_index()
      .rename(columns={'id': 'record_count'})
)


ev_census_df = ev_census_df.sort_values(by=['STATE', 'city', 'Year'])

ev_census_df.head()

,station_name,city,street_address,latitude,longitude,ev_connector_types,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_num,ev_pricing,...,ev_network,ZIP,COUNTYNAME,STATE,state_name,Year,Month,population,median_income,record_count
9372,Anchorage Museum,Anchorage,625 C St,61.2157121227527,-149.884839700095,"[""""J1772""""]",NaN,1,NaN,NaN,...,Non-Networked,99501,Anchorage Municipality,AK,"Anchorage Municipality, Alaska",2023.0,07,289069.0,98152.0,1
9373,Anchorage Point S,Anchorage,4647 Old Seward Hwy,61.1778809870228,-149.867371123044,"[""""TESLA""""]",NaN,1,NaN,Free,...,Non-Networked,99503,Anchorage Municipality,AK,"Anchorage Municipality, Alaska",2023.0,06,289069.0,98152.0,1
34063,EasyPark Susitna Parking Lot,Anchorage,898 W 5th Ave,61.2172294401134,-149.89885532938,"[""""J1772""""]",NaN,1,NaN,NaN,...,Non-Networked,99501,Anchorage Municipality,AK,"Anchorage Municipality, Alaska",2023.0,08,289069.0,98152.0,1
3445,49th State Brewing Company - Tesla Destination,Anchorage,717 W 3rd Ave,61.21963,-149.89592,"[""""TESLA""""]",NaN,1,NaN,Free,...,Tesla Destination,99514,Anchorage Municipality,AK,"Anchorage Municipality, Alaska",2024.0,07,289069.0,98152.0,1
10299,Avis Anchorage,Anchorage,6607 Old Seward Hwy,61.16085,-149.86374,"[""""J1772""""]",NaN,4,NaN,NaN,...,TURNONGREEN,99518,Anchorage Municipality,AK,"Anchorage Municipality, Alaska",2024.0,09,289069.0,98152.0,1


# **5. AFDC Vehicle Registration Data**

[AFDC Vehicle Registration](https://afdc.energy.gov/vehicle-registration)

In [22]:
vehicle_df = pd.read_csv(f"{path}/data/afdc/Vehicle_Registration_Counts_by_State.csv")
vehicle_df = vehicle_df.drop(['Biodiesel', 'Ethanol/Flex (E85)', 'Compressed Natural Gas (CNG)', 'Propane', 'Hydrogen', 'Methanol', 'Gasoline', 'Diesel', 'Unknown Fuel'], axis=1)

state_map = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN',
    'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR',
    'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
vehicle_df.rename(columns={"State": "STATE_NAME"}, inplace = True)
vehicle_df['STATE'] = vehicle_df['STATE_NAME'].map(state_map)

vehicle_df.head()

,STATE_NAME,Year,Electric (EV),Plug-In Hybrid Electric (PHEV),Hybrid Electric (HEV),STATE
0,Alabama,2023,"13,000","5,800","63,300",AL
1,Alaska,2023,"2,700",900,"10,600",AK
2,Arizona,2023,"89,800","25,600","175,700",AZ
3,Arkansas,2023,"7,100","3,200","37,000",AR
4,California,2023,"1,256,600","410,700","1,703,200",CA


# **Merging AFS/Census with Vehicle Registration**

In [23]:
ev_census_vehicle_df = pd.merge( ev_census_df, vehicle_df, how="left", on=["Year","STATE"])
ev_census_vehicle_df[["STATE", "city", "state_name", "Year", "Month", "STATE_NAME"]].head()

,STATE,city,state_name,Year,Month,STATE_NAME
0,AK,Anchorage,"Anchorage Municipality, Alaska",2023.0,07,Alaska
1,AK,Anchorage,"Anchorage Municipality, Alaska",2023.0,06,Alaska
2,AK,Anchorage,"Anchorage Municipality, Alaska",2023.0,08,Alaska
3,AK,Anchorage,"Anchorage Municipality, Alaska",2024.0,07,NaN
4,AK,Anchorage,"Anchorage Municipality, Alaska",2024.0,09,NaN


In [24]:
ev_census_vehicle_df.shape

(102623, 25)

# **Data Cleansing & Renaming AFS/AFDC Data**

In [25]:
ev_census_vehicle_df = ev_census_vehicle_df.drop(['STATE', 'ZIP'], axis=1)

ev_census_vehicle_df = ev_census_vehicle_df.rename(columns={
    'id': 'ID',
    'date_last_confirmed': 'Date',
    'station_name': 'STATION_NAME',
    'city': 'CITY',
    'state': 'STATE',
    'street_address': 'STREET_ADDRESS',
    'zip': 'ZIP',
    'latitude': 'LATITUDE',
    'longitude': 'LONGITUDE',
    'ev_connector_types': 'EV_CONNECTOR_TYPES',
    'ev_level1_evse_num': 'EV_LEVEL1_EVSE_NUM',
    'ev_level2_evse_num': 'EV_LEVEL2_EVSE_CNUM',
    'ev_dc_fast_num': 'EV_DC_FAST_NUM',
    'ev_pricing': 'EV_PRICING',
    'access_days_time': 'ACCESS_DAYS_TIME',
    'facility_type': 'FACILITY_TYPE',
    'ev_network': 'EV_NETWORK',
    'COUNTYNAME': 'COUNTY_NAME',
    'state_name': 'COUNTY_STATE',
    'population': 'POPULATION',
    'median_income': 'MEDIAN_INCOME',
    'State': 'STATE_NAME',
    'Electric (EV)': 'ELECTRIC_VEHICLE_REG_COUNT',
    'Plug-In Hybrid Electric (PHEV)': 'PLUG_IN_HYBRID_VEHICLE_REG_COUNT',
    'Hybrid Electric (HEV)': 'HYBRID_ELECTRIC_REG_COUNT'
})
ev_census_vehicle_df.head()

,STATION_NAME,CITY,STREET_ADDRESS,LATITUDE,LONGITUDE,EV_CONNECTOR_TYPES,EV_LEVEL1_EVSE_NUM,EV_LEVEL2_EVSE_CNUM,EV_DC_FAST_NUM,EV_PRICING,...,COUNTY_STATE,Year,Month,POPULATION,MEDIAN_INCOME,record_count,STATE_NAME,ELECTRIC_VEHICLE_REG_COUNT,PLUG_IN_HYBRID_VEHICLE_REG_COUNT,HYBRID_ELECTRIC_REG_COUNT
0,Anchorage Museum,Anchorage,625 C St,61.2157121227527,-149.884839700095,"[""""J1772""""]",NaN,1,NaN,NaN,...,"Anchorage Municipality, Alaska",2023.0,07,289069.0,98152.0,1,Alaska,"2,700",900,"10,600"
1,Anchorage Point S,Anchorage,4647 Old Seward Hwy,61.1778809870228,-149.867371123044,"[""""TESLA""""]",NaN,1,NaN,Free,...,"Anchorage Municipality, Alaska",2023.0,06,289069.0,98152.0,1,Alaska,"2,700",900,"10,600"
2,EasyPark Susitna Parking Lot,Anchorage,898 W 5th Ave,61.2172294401134,-149.89885532938,"[""""J1772""""]",NaN,1,NaN,NaN,...,"Anchorage Municipality, Alaska",2023.0,08,289069.0,98152.0,1,Alaska,"2,700",900,"10,600"
3,49th State Brewing Company - Tesla Destination,Anchorage,717 W 3rd Ave,61.21963,-149.89592,"[""""TESLA""""]",NaN,1,NaN,Free,...,"Anchorage Municipality, Alaska",2024.0,07,289069.0,98152.0,1,NaN,NaN,NaN,NaN
4,Avis Anchorage,Anchorage,6607 Old Seward Hwy,61.16085,-149.86374,"[""""J1772""""]",NaN,4,NaN,NaN,...,"Anchorage Municipality, Alaska",2024.0,09,289069.0,98152.0,1,NaN,NaN,NaN,NaN


# **6. EV Watts Data**

[EV Watts Data](https://livewire.energy.gov/ds/evwatts/evwatts.public)

In [26]:
evse_df = pd.read_csv(f"{path}/data/evwatts/evwatts.public.evse.csv")
evse_df.head(5)

,evse_id,metro_area,land_use,region,num_ports,charge_level,venue,pricing
0,6034,Undesignated,Metro Area,Mountain,1,DCFC,Corridor,Paid
1,6065,Undesignated,Metro Area,Mountain,1,DCFC,Corridor,Paid
2,6100,"Phoenix-Mesa-Chandler, AZ Metro Area",Metro Area,Mountain,1,DCFC,Undesignated,Free
3,6155,Undesignated,Metro Area,Mountain,1,DCFC,Corridor,Paid
4,1315,"Denver-Aurora-Lakewood, CO Metro Area",Metro Area,Mountain,1,DCFC,Corridor,Free


In [27]:
session_df = pd.read_csv(f"{path}/data/evwatts/evwatts.public.session.csv")
session_df.head(5)

,session_id,evse_id,connector_id,start_datetime,end_datetime,total_duration,charge_duration,energy_kwh,start_soc,end_soc,flag_id
0,11562,5,5,2020-03-01 09:37:44,2020-03-01 11:14:59,1.621389,1.615556,6.170,NaN,NaN,0
1,11563,116,116,2020-03-01 10:02:24,2020-03-01 11:08:14,1.096944,1.089722,5.212,NaN,NaN,0
2,11564,72,72,2020-03-01 10:06:08,2020-03-01 11:08:10,1.033611,1.024722,3.392,NaN,NaN,0
3,11565,1,1,2020-03-01 10:01:12,2020-03-01 11:07:09,1.099722,1.076111,6.521,NaN,NaN,0
4,11566,0,138,2020-02-29 19:09:03,2020-03-01 10:47:16,15.636667,12.658889,80.527,NaN,NaN,0


In [28]:
vehicles_df = pd.read_csv(f"{path}/data/evwatts/evwatts.public.vehicles.csv")
vehicles_df.head(5)

,id,vehicle_type,electrification_level,ownership,state
0,100,PASSENGER CAR,BEV (Battery Electric Vehicle),Fleet,Colorado
1,101,PASSENGER CAR,BEV (Battery Electric Vehicle),Fleet,Colorado
2,102,PASSENGER CAR,BEV (Battery Electric Vehicle),Fleet,Colorado
3,103,PASSENGER CAR,BEV (Battery Electric Vehicle),Fleet,Colorado
4,104,PASSENGER CAR,BEV (Battery Electric Vehicle),Fleet,Colorado


In [29]:
connector_df = pd.read_csv(f"{path}/data/evwatts/evwatts.public.connector.csv")
connector_df.head(5)

,connector_id,evse_id,connector_number,connector_type,power_kw
0,38494,25683,1,J1772,>8 kW
1,38358,27850,1,J1772,>8 kW
2,44144,31875,1,J1772,>8 kW
3,45088,32971,1,J1772,>8 kW
4,9643,5712,1,J1772,<8 kW


# **Merging EV Session with EV Connector**

In [30]:
#Merge connector and session on connector_id
merged_df = session_df.merge(connector_df, on='connector_id', how='left')
merged_df = merged_df.drop(columns='evse_id_x').rename(columns={'evse_id_y': 'evse_id'})
merged_df['start_datetime'] = pd.to_datetime(merged_df['start_datetime'])
merged_df['end_datetime'] = pd.to_datetime(merged_df['end_datetime'])
merged_df['date'] = merged_df['start_datetime'].dt.date
merged_df['hour'] = merged_df['start_datetime'].dt.hour
merged_df['month'] = merged_df['start_datetime'].dt.to_period('M')
merged_df.head()

,session_id,connector_id,start_datetime,end_datetime,total_duration,charge_duration,energy_kwh,start_soc,end_soc,flag_id,evse_id,connector_number,connector_type,power_kw,date,hour,month
0,11562,5,2020-03-01 09:37:44,2020-03-01 11:14:59,1.621389,1.615556,6.170,NaN,NaN,0,5.0,2.0,J1772,<8 kW,2020-03-01,9,2020-03
1,11563,116,2020-03-01 10:02:24,2020-03-01 11:08:14,1.096944,1.089722,5.212,NaN,NaN,0,116.0,2.0,J1772,<8 kW,2020-03-01,10,2020-03
2,11564,72,2020-03-01 10:06:08,2020-03-01 11:08:10,1.033611,1.024722,3.392,NaN,NaN,0,72.0,2.0,J1772,<8 kW,2020-03-01,10,2020-03
3,11565,1,2020-03-01 10:01:12,2020-03-01 11:07:09,1.099722,1.076111,6.521,NaN,NaN,0,1.0,2.0,J1772,<8 kW,2020-03-01,10,2020-03
4,11566,138,2020-02-29 19:09:03,2020-03-01 10:47:16,15.636667,12.658889,80.527,NaN,NaN,0,0.0,1.0,J1772,<8 kW,2020-02-29,19,2020-02


# **Merge with evse data**

In [31]:
merged_df = merged_df.merge(evse_df, on='evse_id', how='left')
merged_df.head()

,session_id,connector_id,start_datetime,end_datetime,total_duration,charge_duration,energy_kwh,start_soc,end_soc,flag_id,...,date,hour,month,metro_area,land_use,region,num_ports,charge_level,venue,pricing
0,11562,5,2020-03-01 09:37:44,2020-03-01 11:14:59,1.621389,1.615556,6.170,NaN,NaN,0,...,2020-03-01,9,2020-03,Undesignated,Undesignated,Middle Atlantic,2.0,L2,Municipal Building,Undesignated
1,11563,116,2020-03-01 10:02:24,2020-03-01 11:08:14,1.096944,1.089722,5.212,NaN,NaN,0,...,2020-03-01,10,2020-03,Undesignated,Metro Area,Middle Atlantic,2.0,L2,Medical or Educational Campus,Undesignated
2,11564,72,2020-03-01 10:06:08,2020-03-01 11:08:10,1.033611,1.024722,3.392,NaN,NaN,0,...,2020-03-01,10,2020-03,"Rochester, NY Metro Area",Metro Area,Middle Atlantic,2.0,L2,Municipal Building,Undesignated
3,11565,1,2020-03-01 10:01:12,2020-03-01 11:07:09,1.099722,1.076111,6.521,NaN,NaN,0,...,2020-03-01,10,2020-03,"Albany-Schenectady-Troy, NY Metro Area",Metro Area,Middle Atlantic,2.0,L2,Municipal Building,Undesignated
4,11566,138,2020-02-29 19:09:03,2020-03-01 10:47:16,15.636667,12.658889,80.527,NaN,NaN,0,...,2020-02-29,19,2020-02,"Albany-Schenectady-Troy, NY Metro Area",Metro Area,Middle Atlantic,2.0,L2,Municipal Building,Undesignated


In [32]:
sample_df = merged_df[merged_df['start_datetime'].dt.year.isin([2022, 2023])]
sample_df['STATE'] = (
    sample_df['metro_area']
    .str.extract(r',\s*([A-Z]{2})(?:\s*-\s*[A-Z]{2})*\s+Metro', expand=False)
)

#sample_df['Year'] = sample_df['start_datetime'].dt.year
#sample_df['Year_Month'] = sample_df['start_datetime'].dt.strftime('%b, %Y')
#sample_df['Month'] = sample_df['start_datetime'].dt.strftime('%m')



In [33]:
print(f"EVSE Record Length : {len(evse_df)}")
print(f"Connector Record Length : {len(connector_df)}")
print(f"Session Record Length : {len(session_df)}")
print(f"Merged Df Record Length : {len(merged_df)}")
print(f"All EVWATTS Df Record Length : {len(sample_df)}")

EVSE Record Length : 36926
Connector Record Length : 50103
Session Record Length : 13937235
Merged Df Record Length : 13937235
All EVWATTS Df Record Length : 7184136


In [34]:
sample_df.head()

,session_id,connector_id,start_datetime,end_datetime,total_duration,charge_duration,energy_kwh,start_soc,end_soc,flag_id,...,hour,month,metro_area,land_use,region,num_ports,charge_level,venue,pricing,STATE
5329935,5889585,10213,2022-01-01 02:03:42,2022-01-01 02:54:29,0.846667,0.842778,5.047,NaN,NaN,0,...,2,2022-01,Undesignated,Metro Area,East South Central,2.0,L2,Retail,Undesignated,NaN
5329937,5889587,10162,2022-01-01 01:25:06,2022-01-01 02:20:51,0.930000,0.922778,6.026,NaN,NaN,0,...,1,2022-01,Undesignated,Metro Area,East South Central,2.0,L2,Retail,Undesignated,NaN
5329938,5889588,10161,2022-01-01 00:42:44,2022-01-01 01:02:08,0.323889,0.320000,1.987,NaN,NaN,0,...,0,2022-01,Undesignated,Metro Area,East South Central,2.0,L2,Business Office,Undesignated,NaN
5344582,5912932,20125,2022-01-01 01:36:35,2022-01-02 17:56:54,40.339167,1.542778,5.735,NaN,NaN,0,...,1,2022-01,"Boston-Cambridge-Newton, MA-NH Metro Area",Metro Area,New England,2.0,L2,Multi-Unit Dwelling,Undesignated,MA
5344586,5912936,20205,2022-01-01 00:49:34,2022-01-01 17:25:35,16.600278,4.815833,14.675,NaN,NaN,0,...,0,2022-01,"Boston-Cambridge-Newton, MA-NH Metro Area",Metro Area,New England,2.0,L2,Multi-Unit Dwelling,Undesignated,MA


In [44]:
cols_to_sum = ['total_duration', 'charge_duration', 'energy_kwh', 'power_kw', 'num_ports']
for c in cols_to_sum:
    sample_df[c] = pd.to_numeric(sample_df[c], errors='coerce')

agg_dict = {c: 'sum' for c in cols_to_sum}
agg_dict.update({
    'session_id': 'count'
})
grouped = (
    sample_df.groupby(['date', 'STATE'], dropna=False)
      .agg(agg_dict)
      .reset_index()
      .rename(columns={'STATE': 'state', 'session_id': 'sessions'})
)
grouped.rename(columns={
    "state": "STATE"
}, inplace=True)
grouped["Year"] = pd.to_datetime( grouped["date"] ).dt.year
grouped['Year_Month'] = pd.to_datetime( grouped['date'] ).dt.strftime('%b, %Y')
grouped['Month'] = pd.to_datetime( grouped['date'] ).dt.strftime('%m')


grouped.head()

,date,STATE,total_duration,charge_duration,energy_kwh,power_kw,num_ports,sessions,Year,Year_Month,Month
0,2022-01-01,AZ,1535.748889,684.635833,6894.701085,0.0,379.0,265,2022,"Jan, 2022",01
1,2022-01-01,CA,308.249444,255.938611,7422.100994,0.0,322.0,322,2022,"Jan, 2022",01
2,2022-01-01,CO,549.503333,391.791389,2719.197000,0.0,274.0,166,2022,"Jan, 2022",01
3,2022-01-01,DC,5009.886111,1121.783889,6695.792935,0.0,540.0,450,2022,"Jan, 2022",01
4,2022-01-01,FL,1748.310833,802.501667,4366.120773,0.0,368.0,201,2022,"Jan, 2022",01


In [45]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7300 entries, 0 to 7299
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             7300 non-null   object 
 1   STATE            6935 non-null   object 
 2   total_duration   7300 non-null   float64
 3   charge_duration  7300 non-null   float64
 4   energy_kwh       7300 non-null   float64
 5   power_kw         7300 non-null   float64
 6   num_ports        7300 non-null   float64
 7   sessions         7300 non-null   int64  
 8   Year             7300 non-null   int32  
 9   Year_Month       7300 non-null   object 
 10  Month            7300 non-null   object 
dtypes: float64(5), int32(1), int64(1), object(4)
memory usage: 599.0+ KB


In [50]:
ev_census_vehicle_df["Year"] = pd.to_numeric( ev_census_vehicle_df["Year"] )
state_map = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN',
    'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR',
    'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
ev_census_vehicle_df['STATE'] = ev_census_vehicle_df['STATE_NAME'].map(state_map)


ev_census_vehicle_df.head()

,STATION_NAME,CITY,STREET_ADDRESS,LATITUDE,LONGITUDE,EV_CONNECTOR_TYPES,EV_LEVEL1_EVSE_NUM,EV_LEVEL2_EVSE_CNUM,EV_DC_FAST_NUM,EV_PRICING,...,Year,Month,POPULATION,MEDIAN_INCOME,record_count,STATE_NAME,ELECTRIC_VEHICLE_REG_COUNT,PLUG_IN_HYBRID_VEHICLE_REG_COUNT,HYBRID_ELECTRIC_REG_COUNT,STATE
0,Anchorage Museum,Anchorage,625 C St,61.2157121227527,-149.884839700095,"[""""J1772""""]",NaN,1,NaN,NaN,...,2023.0,07,289069.0,98152.0,1,Alaska,"2,700",900,"10,600",AK
1,Anchorage Point S,Anchorage,4647 Old Seward Hwy,61.1778809870228,-149.867371123044,"[""""TESLA""""]",NaN,1,NaN,Free,...,2023.0,06,289069.0,98152.0,1,Alaska,"2,700",900,"10,600",AK
2,EasyPark Susitna Parking Lot,Anchorage,898 W 5th Ave,61.2172294401134,-149.89885532938,"[""""J1772""""]",NaN,1,NaN,NaN,...,2023.0,08,289069.0,98152.0,1,Alaska,"2,700",900,"10,600",AK
3,49th State Brewing Company - Tesla Destination,Anchorage,717 W 3rd Ave,61.21963,-149.89592,"[""""TESLA""""]",NaN,1,NaN,Free,...,2024.0,07,289069.0,98152.0,1,NaN,NaN,NaN,NaN,NaN
4,Avis Anchorage,Anchorage,6607 Old Seward Hwy,61.16085,-149.86374,"[""""J1772""""]",NaN,4,NaN,NaN,...,2024.0,09,289069.0,98152.0,1,NaN,NaN,NaN,NaN,NaN


In [47]:
ev_census_vehicle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102623 entries, 0 to 102622
Data columns (total 23 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   STATION_NAME                      102622 non-null  object 
 1   CITY                              102619 non-null  object 
 2   STREET_ADDRESS                    102582 non-null  object 
 3   LATITUDE                          102623 non-null  object 
 4   LONGITUDE                         102623 non-null  object 
 5   EV_CONNECTOR_TYPES                102595 non-null  object 
 6   EV_LEVEL1_EVSE_NUM                752 non-null     object 
 7   EV_LEVEL2_EVSE_CNUM               85068 non-null   object 
 8   EV_DC_FAST_NUM                    19159 non-null   object 
 9   EV_PRICING                        19732 non-null   object 
 10  FACILITY_TYPE                     26140 non-null   object 
 11  EV_NETWORK                        102602 non-null  o

In [51]:
print(f"Length of ev_census_vehicle_df : {len( ev_census_vehicle_df )}")
print(f"Length of grouped : {len( grouped )}")

Length of ev_census_vehicle_df : 102623
Length of grouped : 7300


# **Merging All**

In [52]:


ev_census_vehicle_all_df = pd.merge( ev_census_vehicle_df, grouped, how="left", on=[ "Year","Month", "STATE"])
ev_census_vehicle_all_df = ev_census_vehicle_all_df.rename(columns={
    'total_duration': 'TOTAL_DURATION',
    'charge_duration': 'CHARGE_DURATION',
    'energy_kwh': 'ENERGY_KWH',
    'power_kw': 'POWER_KW',
    'num_ports': 'NUM_PORTS',
    'sessions': 'TOTAL_SESSIONS',
    'Date': 'DATE',
    'Year': 'YEAR'
})
ev_census_vehicle_all_df.drop( ["date",], axis=1)
ev_census_vehicle_all_df.head()


,STATION_NAME,CITY,STREET_ADDRESS,LATITUDE,LONGITUDE,EV_CONNECTOR_TYPES,EV_LEVEL1_EVSE_NUM,EV_LEVEL2_EVSE_CNUM,EV_DC_FAST_NUM,EV_PRICING,...,HYBRID_ELECTRIC_REG_COUNT,STATE,date,TOTAL_DURATION,CHARGE_DURATION,ENERGY_KWH,POWER_KW,NUM_PORTS,TOTAL_SESSIONS,Year_Month
0,Anchorage Museum,Anchorage,625 C St,61.2157121227527,-149.884839700095,"[""""J1772""""]",NaN,1,NaN,NaN,...,"10,600",AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Anchorage Point S,Anchorage,4647 Old Seward Hwy,61.1778809870228,-149.867371123044,"[""""TESLA""""]",NaN,1,NaN,Free,...,"10,600",AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EasyPark Susitna Parking Lot,Anchorage,898 W 5th Ave,61.2172294401134,-149.89885532938,"[""""J1772""""]",NaN,1,NaN,NaN,...,"10,600",AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49th State Brewing Company - Tesla Destination,Anchorage,717 W 3rd Ave,61.21963,-149.89592,"[""""TESLA""""]",NaN,1,NaN,Free,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Avis Anchorage,Anchorage,6607 Old Seward Hwy,61.16085,-149.86374,"[""""J1772""""]",NaN,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
ev_census_vehicle_all_df.shape

(116572, 32)

In [54]:
ev_census_vehicle_df.to_csv(f"{path}/data/final_data/ev_census_vehicle_data.csv")